Data Clean and Merge

In [99]:
import pandas as pd
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [100]:
tips = pd.read_csv('yelp_academic_dataset_tip.csv')
tips = tips.drop('type',axis=1)
tips = tips.drop('likes',axis=1)
tips = tips.drop('user_id',axis=1)
tips = tips.drop('date',axis=1)
tips.head()

,business_id,text
0,5UmKMjUEUNdYWqANhGckJw,Pizza is garbage. Hoagies are excellent.
1,cE27W9VPgO88Qxe4ol6y_g,Don't waste your time.
2,mVHrayjG3uZ_RLHkLj-AMg,Not easy to find. Be sure to put in directions...
3,mVHrayjG3uZ_RLHkLj-AMg,Your GPS will not allow you to find this place...
4,KayYbHCt-RkbGcPdGOThNg,Great drink specials!


In [101]:
tips['text'] = tips['text'].str.lower()
tips['text'] = tips['text'].str.replace('[^\w\s]','')
tips.head()

,business_id,text
0,5UmKMjUEUNdYWqANhGckJw,pizza is garbage hoagies are excellent
1,cE27W9VPgO88Qxe4ol6y_g,dont waste your time
2,mVHrayjG3uZ_RLHkLj-AMg,not easy to find be sure to put in directions ...
3,mVHrayjG3uZ_RLHkLj-AMg,your gps will not allow you to find this place...
4,KayYbHCt-RkbGcPdGOThNg,great drink specials


In [102]:
group = tips.groupby('business_id')
tips = group.aggregate(lambda x: list(x))
tips.head()

,text
business_id,
--0ZoBTQWQra1FxD4rBWmg,[dont ignore the reviews this management compa...
--1emggGHgoG6ipd_RMb-g,"[its a gas station nothing special, the temp o..."
--4Pe8BZ6gj57VFL5mUE8g,"[best customer service thank you michael, ite..."
--5jkZ3-nUPZxUvtcbr8Uw,"[try the greek sampler, fast friendly great gy..."
--BlvDO_RG2yElKu9XA1_g,"[good food, tried the hawaiian fried rice curr..."


In [103]:
tips.reset_index(level=0, inplace=True)
tips.head()

,business_id,text
0,--0ZoBTQWQra1FxD4rBWmg,[dont ignore the reviews this management compa...
1,--1emggGHgoG6ipd_RMb-g,"[its a gas station nothing special, the temp o..."
2,--4Pe8BZ6gj57VFL5mUE8g,"[best customer service thank you michael, ite..."
3,--5jkZ3-nUPZxUvtcbr8Uw,"[try the greek sampler, fast friendly great gy..."
4,--BlvDO_RG2yElKu9XA1_g,"[good food, tried the hawaiian fried rice curr..."


In [104]:
tips.shape

(57495, 2)

In [107]:
for i in tips.index: 
    if (i % 1000==0):
        print i,
    business_words = set(' '.join(tips['text'][i]).split())
    tips['text'][i] = business_words
tips.head()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000


,business_id,text
0,--0ZoBTQWQra1FxD4rBWmg,"{and, ive, code, dont, people, doing, is, gett..."
1,--1emggGHgoG6ipd_RMb-g,"{and, heater, is, beer, nothing, inn, been, it..."
2,--4Pe8BZ6gj57VFL5mUE8g,"{customer, on, and, thank, service, items, mic..."
3,--5jkZ3-nUPZxUvtcbr8Uw,"{cute, just, contemporary, still, its, staff, ..."
4,--BlvDO_RG2yElKu9XA1_g,"{all, nongreasy, chinese, hampton, chicken, in..."


In [108]:
tips.to_csv('business_words.csv',index=False)

In [42]:
vocab_count = ' '.join(data['text'].tolist()).split()

In [43]:
cnt = Counter()
for word in vocab_count:
    cnt[word] += 1

In [48]:
word_counts = pd.DataFrame.from_dict(cnt, orient='index').reset_index()
word_counts.columns = ['Word', 'Word Count']
word_counts.head()

,Word,Word Count
0,woods,40
1,hanging,472
2,woody,39
3,gae,8
4,canes,92


In [46]:
for i in range(len(vocab_count)):
    word = vocab_count[i] 
    word_count = cnt[word]
    if word_count < 5:
        del cnt[word] 
    else:
        pass

In [33]:
vocab = list(cnt)

In [34]:
sentiment_list = []
not_modeled_list = []
for word in vocab:
    try:
        sentiment = determine_coefficient(word)
        sentiment_list.append(sentiment)
    except:
        not_modeled_list.append(word)
vocab = set(vocab) - set(not_modeled_list)    
vocab = list(vocab)

In [35]:
sentiment_df = pd.DataFrame({'Word': vocab,'Sentiment': sentiment_list})
sentiment_df.head()

,Sentiment,Word
0,1.145606,raining
1,0.275232,vang
2,0.702242,ioane
3,-0.213470,yellow
4,0.344637,factory


In [38]:
sentiment_df.to_csv('sentiment_vocab_lookup.csv',index=False)

In [9]:
glove_home = '/Users/Laura/GoogleDrive/NYU/NLP/'
words_to_load = 500000

import numpy as np

with open(glove_home + 'glove.6B.50d.txt') as f:
    loaded_embeddings = np.zeros((words_to_load, 50))
    words = {}
    ordered_words = []
    for i, line in enumerate(f):
        if i >= words_to_load: 
            break
        
        s = line.split()
        loaded_embeddings[i, :] = np.asarray(s[1:])
        words[s[0]] = i
        ordered_words.append(s[0])
seed_pos = ['good', 'great', 'awesome', 'like', 'love']
seed_neg = ['bad', 'awful', 'terrible', 'hate', 'dislike']
seed_pos_indices = [words[seed] for seed in seed_pos]
seed_neg_indices = [words[seed] for seed in seed_neg]
seed_pos_mat = loaded_embeddings[seed_pos_indices]
seed_neg_mat = loaded_embeddings[seed_neg_indices]
from sklearn.metrics.pairwise import cosine_similarity

def determine_coefficient(candidate_word):
    candidate = loaded_embeddings[words[candidate_word]]
    pos_sim = np.sum([cosine_similarity(candidate, reference) for reference in seed_pos_mat])
    neg_sim = np.sum([cosine_similarity(candidate, reference) for reference in seed_neg_mat])
    return pos_sim - neg_sim  
determine_coefficient('abhorrent')

KeyboardInterrupt: 

In [50]:
data = pd.merge(sentiment_df,word_counts, on='Word')

In [51]:
data.head()

,Sentiment,Word,Word Count
0,1.145606,raining,50
1,0.275232,vang,10
2,0.702242,ioane,5
3,-0.213470,yellow,369
4,0.344637,factory,186


In [53]:
data['Log Word Count'] = data['Word Count'].apply(lambda x:np.log(x))
data.head()

,Sentiment,Word,Word Count,Log Word Count
0,1.145606,raining,50,3.912023
1,0.275232,vang,10,2.302585
2,0.702242,ioane,5,1.609438
3,-0.213470,yellow,369,5.910797
4,0.344637,factory,186,5.225747


In [54]:
data = data.drop('Word Count',axis=1)
data.head()

,Sentiment,Word,Log Word Count
0,1.145606,raining,3.912023
1,0.275232,vang,2.302585
2,0.702242,ioane,1.609438
3,-0.213470,yellow,5.910797
4,0.344637,factory,5.225747


In [55]:
data.to_csv('word_sentiment_freq.csv', index=False)

In [4]:
good_states = ['AZ','NV','NC','QC','PA','WI']
businesses = businesses.loc[businesses['state'].isin(good_states)]
businesses = businesses.drop('state',axis=1)
businesses = businesses.reset_index(drop=True)
businesses.head()

,business_id,categories
0,5UmKMjUEUNdYWqANhGckJw,"Fast Food,Restaurants"
1,UsFtqoBl7naz8AVUBZMjQQ,Nightlife
2,cE27W9VPgO88Qxe4ol6y_g,"Active Life,Mini Golf,Golf"
3,mVHrayjG3uZ_RLHkLj-AMg,"Bars,American (New),Nightlife,Lounges,Restaurants"
4,mYSpR_SLPgUVymYOvTQd_Q,"Active Life,Golf"


In [3]:
business_fields = ['business_id','categories','state']
businesses = pd.read_csv('yelp_academic_dataset_business.csv', skipinitialspace=True, usecols=business_fields)
businesses.head()

,business_id,categories,state
0,5UmKMjUEUNdYWqANhGckJw,"Fast Food,Restaurants",PA
1,UsFtqoBl7naz8AVUBZMjQQ,Nightlife,PA
2,cE27W9VPgO88Qxe4ol6y_g,"Active Life,Mini Golf,Golf",PA
3,mVHrayjG3uZ_RLHkLj-AMg,"Bars,American (New),Nightlife,Lounges,Restaurants",PA
4,mYSpR_SLPgUVymYOvTQd_Q,"Active Life,Golf",PA


In [26]:
categories_list = businesses['categories'].tolist()
broad_categories = []
for i in businesses.index:
    category = str(categories_list[i])
    if 'Restaurants' in category or 'Food' in category:
        new_category = 'Eateries'
    elif 'Shopping' in category or  'Hotel' in category or 'Nightlife' in category or 'Entertainment' in category:
        new_category = 'Lifestyle & Entertainment'
    elif 'Medical' in category or 'Active' in category or 'Fitness' in category or 'Beauty' in category:
        new_category = 'Health & Beauty'
    elif 'Auto' in category:
        new_category = 'Auto'
    else:
        new_category = 'Others'
    broad_categories.append(new_category)
businesses['broad_categories'] = broad_categories 
businesses = businesses.drop('categories',axis=1)
businesses.head()

,business_id,broad_categories
0,5UmKMjUEUNdYWqANhGckJw,Eateries
1,UsFtqoBl7naz8AVUBZMjQQ,Lifestyle & Entertainment
2,cE27W9VPgO88Qxe4ol6y_g,Health & Beauty
3,mVHrayjG3uZ_RLHkLj-AMg,Eateries
4,mYSpR_SLPgUVymYOvTQd_Q,Health & Beauty


In [341]:
sentiment_average_list = []
for i in data.index:
    if (i % 1000==0):
        print i
    sentiment_each_word = []
    text = data['text'][i]
    words = text.split()
    for w in words:
        sentiment = list(sentiment_df.loc[sentiment_df['Word'] == w]['Sentiment'])
        if len(sentiment) > 0:
            sentiment_each_word.append(sentiment)
        else:
            pass
    sentiment_average = np.mean(sentiment_each_word)
    sentiment_average_list.append(sentiment_average)
data['average_sentiment'] = sentiment_average_list

0
1000
2000


KeyboardInterrupt: 

In [344]:
len(sentiment_average_list)

2505

In [345]:
temp = sentiment_average_list + [0]*(len(data) - len(sentiment_average_list))
len(temp)

639079

In [419]:
data['average_sentiment'] = temp

In [429]:
#data = data.drop('stars',axis=1)
#date = []
#for i in data.index:
#   s= '-'
#   seq = (str(data.year[i]),str(data.month[i]),'01')
#   date.append(s.join(seq))
#data['date'] = date
#data = data.drop('year',axis=1)
#data = data.drop('month',axis=1)

In [436]:
data.head()

,business_id,broad_categories,text,average_sentiment,date
0,5UmKMjUEUNdYWqANhGckJw,Eateries,pizza is garbage hoagies are excellent,0.393774,2016-07-01
1,cE27W9VPgO88Qxe4ol6y_g,Health & Beauty,dont waste your time,0.273090,2013-04-01
2,mVHrayjG3uZ_RLHkLj-AMg,Eateries,not easy to find be sure to put in directions ...,0.251693,2016-02-01
3,mVHrayjG3uZ_RLHkLj-AMg,Eateries,your gps will not allow you to find this place...,0.207499,2013-01-01
4,KayYbHCt-RkbGcPdGOThNg,Eateries,great drink specials,0.725087,2013-12-01


In [437]:
data.to_csv('sentiment_to_merge.csv',index=False)

In [464]:
data.shape

(639079, 6)

In [465]:
stars = [1,2,3,4,5]
#years = data['year'].unique()
#months = data['month'].unique()
categories = data['broad_categories'].unique()

In [466]:
combos = []
for a in stars:
    for b in categories: 
        combos = combos + [a,b]
combos = pd.Series(combos).values.reshape(len(combos)/2,2)
print combos

[[1 'Eateries']
 [1 'Health & Beauty']
 [1 'Others']
 [1 'Auto']
 [1 'Lifestyle & Entertainment']
 [2 'Eateries']
 [2 'Health & Beauty']
 [2 'Others']
 [2 'Auto']
 [2 'Lifestyle & Entertainment']
 [3 'Eateries']
 [3 'Health & Beauty']
 [3 'Others']
 [3 'Auto']
 [3 'Lifestyle & Entertainment']
 [4 'Eateries']
 [4 'Health & Beauty']
 [4 'Others']
 [4 'Auto']
 [4 'Lifestyle & Entertainment']
 [5 'Eateries']
 [5 'Health & Beauty']
 [5 'Others']
 [5 'Auto']
 [5 'Lifestyle & Entertainment']]


In [467]:
combos_dataframe = pd.DataFrame(columns=['stars','broad_categories','word_count_dict'])

In [468]:
combos_dataframe.head()

,stars,broad_categories,word_count_dict


In [471]:
cols=['stars','broad_categories','word_count_dict']
for combo in combos[:100]: 
    star = combo[0]
    category = combo[1]
    data_slice = data.loc[(data.stars == star) & (data.broad_categories == category),['text']]
    combo_vocab = ' '.join(data_slice['text'].tolist()).split()
    cnt = Counter()
    for word in combo_vocab:
        cnt[word] += 1
    entry = [star,category,cnt]
    temp_dataframe = pd.DataFrame([entry],columns=cols)
    combos_dataframe = combos_dataframe.append(temp_dataframe,ignore_index=True)
combos_dataframe

,stars,broad_categories,word_count_dict
0,1.0,Eateries,"{u'limited': 1, u'all': 6, u'chinese': 3, u'ma..."
1,1.0,Health & Beauty,"{u'limited': 1, u'all': 3, u'beware': 1, u'man..."
2,1.0,Others,"{u'neighbors': 3, u'limited': 1, u'all': 18, u..."
3,1.0,Auto,"{u'all': 3, u'manager': 1, u'go': 3, u'follow'..."
4,1.0,Lifestyle & Entertainment,"{u'limited': 2, u'all': 1, u'liar': 3, u'switc..."
5,2.0,Eateries,"{u'yellow': 1, u'four': 4, u'prices': 70, u'ha..."
6,2.0,Health & Beauty,"{u'limited': 1, u'jokedont': 1, u'demand': 1, ..."
7,2.0,Others,"{u'saves': 2, u'desirable': 1, u'felt': 1, u'f..."
8,2.0,Auto,"{u'todays': 1, u'yellow': 1, u'enoughdo': 1, u..."
9,2.0,Lifestyle & Entertainment,"{u'limited': 1, u'ciao': 1, u'consolidated': 1..."


In [473]:
combos_dataframe.to_csv('word_counts_in_combos.csv',index=0)